In [2]:
import os
import json
import re
import csv
from scipy.stats import chi2_contingency
import numpy as np
import xlrd
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import  metrics
import random
random.seed(1)
np.random.seed(1)

/Users/linliu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
Train_Fake_Data = []
Train_True_Data = []
Test_Fake_Data = []
Test_True_Data = []

In [4]:
f = open("./data/Fake_train")
for line in f.readlines():
    Train_Fake_Data.append(line)
f.close()

f = open("./data/True_train")
for line in f.readlines():
    Train_True_Data.append(line)
f.close()

f = open("./data/Fake_test")
for line in f.readlines():
    Test_Fake_Data.append(line)
f.close()

f = open("./data/True_test")
for line in f.readlines():
    Test_True_Data.append(line)
f.close()

In [5]:
def get_feature(True_data=None, Fake_data=None):
    features_word = ['all', 'propaganda', 'persuasive','sentiment', 'subjectivity', 'technical']
    all_feature_words_list = []
    for i, word in enumerate(features_word):
        true_path = './result_fake_true_after_reduce_stopwords/' + 'true_' + word + '_words.csv'
        fake_path = './result_fake_true_after_reduce_stopwords/' + 'fake_' + word + '_words.csv'
        true_words = open(true_path, encoding='utf-8', mode='r')
        true_words_list = []
        true_words = true_words.readlines()
        for line in true_words:
            line = line.strip('\n\r').split(',')
            # print(line[0])
            true_words_list.append(line[0])
        all_feature_words_list.append(true_words_list)

        fake_words = open(fake_path, encoding='utf-8', mode='r')
        fake_words_list = []
        fake_words = fake_words.readlines()
        for line in fake_words:
            line = line.strip('\n\r').split(',')
            fake_words_list.append(line[0])
            # print(line[0])
        all_feature_words_list.append(fake_words_list)

    print(len(all_feature_words_list))

    all_data = []
    for line in True_data[:]:
        one_data = [0, ]
        line = line.split(' ')
        for feature in all_feature_words_list:
            num = 0
            for word in line:
                if word in feature:
                    num += 1
            one_data.append(num)
        # print(one_data)
        all_data.append(one_data)

    for line in Fake_data[:]:
        one_data = [1, ]
        line = line.split(' ')
        for feature in all_feature_words_list:
            num = 0
            for word in line:
                if word in feature:
                    num += 1
            one_data.append(num)
        all_data.append(one_data)
    # print(all_data[:3])
    '''
    [0, 2, 0, 16, 5, 0, 0, 2, 0, 1, 0]
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    [1, 1, 0, 3, 0, 1, 0, 2, 0, 0, 0]
    [1, 1, 0, 3, 1, 0, 0, 0, 0, 0, 0]

    '''

    return all_data

In [6]:
train_data = get_feature(Train_True_Data, Train_Fake_Data)
test_data = get_feature(Test_True_Data, Test_Fake_Data)

random.shuffle(train_data)
random.shuffle(test_data)

12
12


# GBDT

In [33]:
data = np.array(train_data) 
# print(data)
x = data[:, 1:] 
y = data[:, 0].astype(int)  

In [34]:
test_data = np.array(test_data)
test_x = test_data[:, 1:] 
test_y = test_data[:, 0].astype(int) 

In [35]:
num_train = x.shape[0]
num_each_part = num_train//5
print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
test_accuracy_list = list()
test_f1_list = list()
for i in range(5):
    print(num_each_part*i,num_each_part*(i+1))
    train_x = x[num_each_part*i:num_each_part*(i+1)]
    train_y = y[num_each_part*i:num_each_part*(i+1)]
    gbdt = GradientBoostingClassifier(max_depth=6,
                                      random_state=10,
                                      min_samples_split=10,
                                      learning_rate=0.01,
                                      n_estimators=200,
                                      subsample=0.85)
    gbdt.fit(train_x,train_y)
    train_score_rbf = gbdt.score(train_x, train_y)
    print("The train accuracy score of GBDT is : %f" % train_score_rbf)
    test_score_rbf = gbdt.score(test_x, test_y)
    print("The test accuracy score of GBDT is : %f" % test_score_rbf)
    predict_y = gbdt.predict(test_x)
    prob_y = gbdt.predict_proba(test_x)[:,1]
    from sklearn.metrics import confusion_matrix,f1_score
    f1 = f1_score(test_y, predict_y, average='binary')
    print("f1:", f1 )
    test_accuracy_list.append(test_score_rbf)
    test_f1_list.append(f1)
print("avg of accuracy of five-fold cross validation is %f"%(np.mean(np.asarray(test_accuracy_list))))
print("avg of f1 of five-fold cross validation is %f"%(np.mean(np.asarray(test_f1_list))))

num of train data is 4814 and each of part(1/5) is 962 
0 962
The train accuracy score of GBDT is : 0.900208
The test accuracy score of GBDT is : 0.764925
f1: 0.6911764705882353
962 1924
The train accuracy score of GBDT is : 0.913721
The test accuracy score of GBDT is : 0.744403
f1: 0.6513994910941475
1924 2886
The train accuracy score of GBDT is : 0.887734
The test accuracy score of GBDT is : 0.755597
f1: 0.6614987080103358
2886 3848
The train accuracy score of GBDT is : 0.901247
The test accuracy score of GBDT is : 0.746269
f1: 0.6650246305418719
3848 4810
The train accuracy score of GBDT is : 0.903326
The test accuracy score of GBDT is : 0.727612
f1: 0.6490384615384616
avg of accuracy of five-fold cross validation is 0.747761
avg of f1 of five-fold cross validation is 0.663628


# add wordnet

In [36]:
f=open('./extend_dict.json',"r")
for line in f:
    extend_dict=json.loads(line)
f.close()

In [37]:
def data_extend(data):
    res_data = []
    for line in data:
        new_line = []
        for word in line.split(' '):
            new_line.append(word)
            if(word in extend_dict):
                new_line.extend(extend_dict[word])
        res_data.append(' '.join(new_line))
    return res_data

In [38]:
Train_Fake_Data_ex = data_extend(Train_Fake_Data)
Train_True_Data_ex = data_extend(Train_True_Data)
Test_Fake_Data_ex = data_extend(Test_Fake_Data)
Test_True_Data_ex = data_extend(Test_True_Data)

In [39]:
def add_feature(True_data=None, Fake_data=None):
    features_word = ['all_extend']
    all_feature_words_list = []
    for i, word in enumerate(features_word):
        true_path = './result_fake_true/' + 'true_' + word + '_words.csv'
        fake_path = './result_fake_true/' + 'fake_' + word + '_words.csv'
        true_words = open(true_path, encoding='utf-8', mode='r')
        true_words_list = []
        true_words = true_words.readlines()
        for line in true_words:
            line = line.strip('\n\r').split(',')
            # print(line[0])
            true_words_list.append(line[0])
        all_feature_words_list.append(true_words_list)

        fake_words = open(fake_path, encoding='utf-8', mode='r')
        fake_words_list = []
        fake_words = fake_words.readlines()
        for line in fake_words:
            line = line.strip('\n\r').split(',')
            fake_words_list.append(line[0])
            # print(line[0])
        all_feature_words_list.append(fake_words_list)

    print(len(all_feature_words_list))

    all_data = []
    for line in True_data[:]:
        one_data = [0, ]
        line = line.split(' ')
        for feature in all_feature_words_list:
            num = 0
            for word in line:
                if word in feature:
                    num += 1
            one_data.append(num)
        # print(one_data)
        all_data.append(one_data)

    for line in Fake_data[:]:
        one_data = [1, ]
        line = line.split(' ')
        for feature in all_feature_words_list:
            num = 0
            for word in line:
                if word in feature:
                    num += 1
            one_data.append(num)
        all_data.append(one_data)
    # print(all_data[:3])
    '''
    [0, 2, 0, 16, 5, 0, 0, 2, 0, 1, 0]
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    [1, 1, 0, 3, 0, 1, 0, 2, 0, 0, 0]
    [1, 1, 0, 3, 1, 0, 0, 0, 0, 0, 0]

  
    '''

    return all_data

In [40]:
train_data = get_feature(Train_True_Data, Train_Fake_Data)
test_data = get_feature(Test_True_Data, Test_Fake_Data)

train_add = add_feature(Train_True_Data_ex, Train_Fake_Data_ex)
test_add = add_feature(Test_True_Data_ex, Test_Fake_Data_ex)

12
12
2
2


In [41]:
def merge_data(data,add):
    res = []
    for i in range(len(data)):
        data[i].extend(add[i][1:])
        res.append(data[i])
    return res

In [42]:
train = merge_data(train_data,train_add)
test = merge_data(test_data,test_add)

In [43]:
data = np.array(train_data) 
np.random.shuffle(data)
# print(data)
x = data[:, 1:]  
y = data[:, 0].astype(int) 

In [44]:
test_data = np.array(test_data)
test_x = test_data[:, 1:] 
test_y = test_data[:, 0].astype(int) 

In [45]:
num_train = x.shape[0]
num_each_part = num_train//1
print("num of train data is %s and each of part(1/5) is %s "%(num_train,num_each_part))
test_accuracy_list = list()
test_f1_list = list()
i=0
print(num_each_part*i,num_each_part*(i+1))
train_x = x[num_each_part*i:num_each_part*(i+1)]
train_y = y[num_each_part*i:num_each_part*(i+1)]
gbdt = GradientBoostingClassifier(max_depth=6,
                                  random_state=10,
                                  min_samples_split=10,
                                  learning_rate=0.01,
                                  n_estimators=200,
                                  subsample=0.85)
gbdt.fit(train_x,train_y)
train_score_rbf = gbdt.score(train_x, train_y)
print("The train accuracy score of GBDT is : %f" % train_score_rbf)
test_score_rbf = gbdt.score(test_x, test_y)
print("The test accuracy score of GBDT is : %f" % test_score_rbf)
predict_y = gbdt.predict(test_x)

num of train data is 4814 and each of part(1/5) is 4814 
0 4814
The train accuracy score of GBDT is : 0.845243
The test accuracy score of GBDT is : 0.759328


In [51]:
true_positive = np.logical_and((predict_y==test_y),test_y)

sentence_target = np.asarray(Test_True_Data)[true_positive]


In [52]:

def load_dict(dict_path):
    word_set = set()
    dict_text = open(dict_path, encoding='utf-8', mode='r')
    for line in dict_text:
        if line.strip()!="":
            word =line.split(",")[0]
            word_set.add(word)
    return word_set



def find_word_in_dicts(sentence_target):
    dict_names = ['persuasive','sentiment','propaganda','subjectivity','technical','all']
    dict_path="./result_fake_true_after_reduce_stopwords/"
    dict_set_list = [load_dict(dict_path+"true_"+name+ "_words.csv").union(load_dict(dict_path+"fake_"+name+ "_words.csv")) for name in dict_names]
    word_dicts_result = dict()
    for sentence in sentence_target:
        sentence = sentence.replace(":"," ").replace("\t"," ")
        word_set = set(sentence.strip().lower().split(' '))
        for word in word_set:
            for index, dict_set in enumerate(dict_set_list):
                if word in dict_set:
                    if sentence in word_dicts_result:
                        word_dicts_result[sentence].add(word+"^"+dict_names[index])
                    else:
                        word_dicts_result[sentence] = set([word+"^"+dict_names[index]])
    return word_dicts_result






word_dicts = find_word_in_dicts(sentence_target)

In [53]:
word_dicts

{'an absurd and wrong claim phil puckett voted no on sending the epa a message that theyve gone too far not once but twice\n': {'claim^all',
  'claim^propaganda',
  'claim^subjectivity',
  'claim^technical',
  'wrong^all',
  'wrong^propaganda',
  'wrong^sentiment',
  'wrong^subjectivity'},
 'jennifer anistons diva set behavior  wont eat lunch with the cast luxury suv and private trailer miles away  jennifer aniston is an alist movie star and radaronlinecom has exclusively learned thats exactly what she acts like on set  while in connecticut filming the untitled elmore leonard project with other famous faces like isla fisher tim robbins and will forte a source close to the set says the former friends actress doesnt socialize with any of the cast or crew and purposely keeps herself removed as much as possible  every day at lunch the entire cast and crew head to a cafeteria facility where they all sit down and eat lunch together for well over an hour the insider exclusively told radaronli

In [ ]:
with open('week9_true_fake_news_result','w') as fout:
    for k, v in word_dicts.items():
        fout.write(k + ":" + "\t".join(v)+"\n")